In [3]:
pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/b9/81/63817fa31d617e0a255125187fc3623dcc3ff5397f1f982ef840cb182056/typed_ast-1.5.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/94/ef/31bfa8456e01ff13d8d98bdbc80ab2e592c830e52ccaff62c35d5f890357/ruamel_yaml_conda-0.15.80.tar.gz
Note: you may need to restart the kernel to use updated packages.


    ERROR: Complete output from command python setup.py egg_info:
    ERROR: Traceback (most recent call last):
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-v1o4ko7j\ruamel-yaml-conda\ruamel_yaml\__init__.py", line 21, in <module>
        from .main import *                               # NOQA
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-v1o4ko7j\ruamel-yaml-conda\ruamel_yaml\main.py", line 12, in <module>
        import ruamel.yaml
    ModuleNotFoundError: No module named 'ruamel'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-v1o4ko7j\ruamel-yaml-conda\setup.py", line 14, in <module>
        import ruamel_yaml  # NOQA
      File "C:\Users\kenda\AppData\Local\Temp\pip-install-v1o4ko7j\ruamel-yaml-conda\ruamel_yaml\__init__.py", line 23, in <module>
        from ruamel_yaml.main import *

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
import re

from top2vec import Top2Vec


# nltk.download('punkt')
# from nltk.tokenize import word_tokenize

# Topic Modeling Using Top2Vec

Load NYT Data for 2020 and 2021

In [5]:
nyt2020 = pd.read_csv("data/NYTData2020.csv")
nyt2021 = pd.read_csv("data/NYTData2021.csv")

nyt = nyt2021.append(nyt2020)

Take NYT review summaries and train a Top2Vec Model

https://pypi.org/project/top2vec/

In [24]:
documents = nyt['summary_short'].tolist()

model = Top2Vec(documents, speed="deep-learn", workers=8,
                embedding_model = "universal-sentence-encoder")

2022-01-06 20:07:20,990 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2022-01-06 20:07:21,096 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2022-01-06 20:07:27,006 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2022-01-06 20:07:27,387 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2022-01-06 20:07:32,032 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2022-01-06 20:07:32,124 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


Print Number of Topics Defined by Model

In [37]:
model.get_num_topics()

3

How many documents are most similar to each topic?

In [38]:
topic_sizes, topic_nums = model.get_topic_sizes()

In [39]:
topic_sizes

array([929, 413,  54], dtype=int64)

Define Top Words for Each Topic

In [32]:
topic_words, word_scores, topic_nums = model.get_topics(model.get_num_topics())

In [33]:
topic_words

array([['comedy', 'drama', 'thriller', 'director', 'movie', 'film',
        'documentary', 'this', 'netflix', 'that', 'into', 'plays', 'it',
        'story', 'the', 'her', 'and', 'as', 'of', 'from', 'its', 'stars',
        'but', 'family', 'is', 'with', 'about', 'for', 'an', 'to', 'in',
        'feature', 'his', 'who', 'at', 'two', 'new', 'by', 'on',
        'follows', 'young', 'are', 'life', 'woman', 'up', 'their'],
       ['documentary', 'thriller', 'film', 'drama', 'that', 'this',
        'movie', 'comedy', 'about', 'it', 'director', 'life', 'into',
        'from', 'story', 'the', 'her', 'of', 'and', 'as', 'but', 'who',
        'woman', 'netflix', 'for', 'an', 'with', 'feature', 'its', 'is',
        'to', 'on', 'two', 'at', 'by', 'family', 'in', 'up', 'his',
        'are', 'plays', 'new', 'stars', 'their', 'young', 'follows'],
       ['documentary', 'netflix', 'movie', 'thriller', 'film', 'comedy',
        'her', 'drama', 'this', 'that', 'director', 'it', 'from', 'as',
        'its'

-----

# Topic Modeling with OMDB Plots

Load in OMDB Data

In [15]:
omdb2020 = pd.read_csv("data/OMDBData2020.csv")
omdb2020.Plot = omdb2020.Plot.fillna(value="")

omdb2021 = pd.read_csv("data/OMDBData2021.csv")
omdb2021.Plot = omdb2021.Plot.fillna(value="")

omdb = omdb2020.append(omdb2021)

Train TFIDF Vectorizer

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

tfidf_vectorizer_NMF = TfidfVectorizer(max_features = 20000, # only top 5k by freq
                                       lowercase = True, # drop capitalization
                                       ngram_range = (1,2), # include up to 2-grams, we can make this only 1 if needed
                                       min_df=1,  # note: absolute count of doc
                                       token_pattern = r'\b[a-z]{3,12}\b',   # remove short, non-word-like terms
                                       stop_words='english') # default English stopwords

tfidf_documents_NMF = tfidf_vectorizer_NMF.fit_transform(omdb["Plot"])
feature_names_NMF = tfidf_vectorizer_NMF.get_feature_names_out()

Train NMF for n = 2-10 topics.

In [23]:
from sklearn.decomposition import NMF
for num_topics in range(2, 11):
    nmf = NMF(
        n_components=num_topics,
        init='nndsvd', 
        random_state = 42
        )
    W = nmf.fit_transform(tfidf_documents_NMF)
    H = nmf.components_
    top_components = np.argsort(-H)[:, :10]
    
    topics = []
    for t in top_components:
        topic_words = []
        for w in t:
            topic_words.append(feature_names_NMF[w])
        topics.append(topic_words)
        
    print(f"{num_topics} Topics:")
    print(topics)
    print("\n")
        

2 Topics:
[['family', 'young', 'old', 'father', 'year', 'woman', 'year old', 'home', 'mother', 'finds'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'city', 'love']]


3 Topics:
[['family', 'young', 'old', 'father', 'woman', 'year', 'home', 'year old', 'mother', 'finds'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'city', 'love'], ['school', 'high', 'high school', 'students', 'senior', 'team', 'student', 'school football', 'school students', 'football']]


4 Topics:
[['family', 'young', 'father', 'woman', 'home', 'mother', 'son', 'wife', 'daughter', 'house'], ['life', 'new', 'film', 'story', 'world', 'documentary', 'york', 'new york', 'love', 'city'], ['school', 'high', 'high school', 'students', 'team', 'senior', 'student', 'school football', 'school students', 'football'], ['year', 'old', 'year old', 'old girl', 'finds', 'love', 'old daughter', 'girl', 'best', 'friend']]


5 Topics:
[['family', 'father', 'young', 'wom

------